<html>
    <h1> Decision Tree
        </h1>
        <p> <b> Min Khant Soe</b> </p>
        <p> ID: 122277 </p>
        <p> ^_^ </p>
    </html>

<html>
    <h3> === Task ==
        </h3>
        <p>  Modify the Decision Tree scratch code in our lecture such that:</p>
        <p> 1) Modify the scratch code so it can accept an hyperparameter max_depth, in which it will continue create the tree until max_depth is reached. </p>
        <p> 2) Put everything into a class DecisionTree. It should have at least two methods, fit(), and predict() </p>
        <p> 3) Load the iris data and try with your class </p>
    </html>

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

In [6]:
class DecisionTree:

    def fit(self, X, y, max_depth = 15):
        self.max_depth = max_depth
        self.n_classes_ = len(set(y))
        self.n_features_ = X.shape[1]
        self.tree_ = self._grow_tree(X, y)
        
    
    def predict(self, X):
        for inputs in X:
            node = self.tree_
            while node.left:
                if inputs[node.feature_index] < node.threshold:
                    node = node.left
                else:
                    node = node.right
        return [node.predicted_class]


    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)
        if depth < self.max_depth:

            m = y.size
            num_parent = [np.sum(y == c) for c in range(self.n_classes_)]
            best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
            best_idx, best_thr = None, None
            
            for idx in range(self.n_features_):
                thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
                num_left = [0] * self.n_classes_
                num_right = num_parent.copy()
                
                for i in range(1, m):
                    c = classes[i - 1]
                    num_left[c] += 1
                    num_right[c] -= 1
                    gini_left = 1.0 - sum(
                        (num_left[x] / i) ** 2 for x in range(self.n_classes_)
                    )
                    gini_right = 1.0 - sum(
                        (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes_)
                    )
                    gini = (i * gini_left + (m - i) * gini_right) / m
                    
                    if thresholds[i] == thresholds[i - 1]:
                        continue
                    if gini < best_gini:
                        best_gini = gini
                        best_idx = idx
                        best_thr = (thresholds[i] + thresholds[i - 1]) / 2
                    
                idx, thr = best_idx, best_thr
            
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

In [7]:
dataset = load_iris()
X, y = dataset.data, dataset.target

In [8]:
Model = DecisionTree()
Model.fit(X,y,max_depth=10)
Pred = Model.predict([[1, 2, 10.5, 5.5, 3]])
print(Pred)

[2]
